In [98]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense, Problem
import pandas as pd

Retrieving covariance and expected return data

In [99]:
covar = pd.read_csv('49StocksCovariance.csv')
er = pd.read_csv('SP500_preparation.csv')
name = pd.read_csv('SP500_preparation.csv',index_col= [0])
names = name.index
s = names.size

covar.reset_index()
x = covar.values
y = er.loc[0:s,'Return']



In [100]:
names

Index(['ADBE', 'A', 'ALB', 'ALGN', 'ALL', 'AXP', 'AME', 'APH', 'AON', 'AMAT',
       'APTV', 'ANET', 'ATO', 'BIO', 'BA', 'CDNS', 'CCL', 'CE', 'CNP', 'CI',
       'CTSH', 'CPRT', 'ELV', 'FMC', 'FTV', 'HON', 'ISRG', 'LRCX', 'LW', 'LYV',
       'MAR', 'MA', 'MCD', 'MTD', 'MCHP', 'MU', 'MSFT', 'MHK', 'MPWR', 'MSCI',
       'NVDA', 'NVR', 'PKG', 'PYPL', 'PGR', 'RSG', 'ROK', 'ROL', 'ROP', 'RCL',
       'SNPS', 'TTWO', 'TER', 'UNH', 'VRSN', 'V'],
      dtype='object')

In [101]:


temp =[]
for i in covar.index:
    for j in covar.index:
        temp += [[i,j,x[i,j]]]



In [102]:
simp = pd.DataFrame(temp,columns=['r','c','covariance'])#simplyfying dataset
simp.set_index(['r','c'])
simp

,r,c,covariance
0,0,0,0.006185
1,0,1,0.002538
2,0,2,-0.000814
3,0,3,0.001406
4,0,4,-0.001185
...,...,...,...
3131,55,51,0.000091
3132,55,52,0.001068
3133,55,53,0.000896
3134,55,54,0.000764


In [103]:
m = Container()

i = Set(m,name= "i",records=covar.index)
j = Set(m,"j",domain=i,records=covar.index)

cv = Parameter(m,"cv",domain=[i,j])
cv.setRecords(simp)
ret= Parameter(m,"ret",domain=[i],records=y)

#decision variable weight
w = Variable(m,name="w",domain=[i],type="Positive",description="Weight of each accet")
#constraints
ub = Equation(m,name="ub")
ub[...] = Sum(i,w[i]) <= 1
#minimum expected return constraint
expr = Equation(m,name="expr")
expr[...] = Sum((i),w[i]*ret[i]) >= 0.05

#objective function
l = 1
obj = l*Sum((i,j),w[i]*w[j]*cv[i,j]) - (1-l)*Sum((i),w[i]*ret[i])

portfolio = Model(
    m,
    name="portfolio",
    equations=[ub,expr],
    problem="NLP",
    sense=Sense.MIN,
    objective=obj,
)
import sys

portfolio.solve(output=sys.stdout)


--- Job _job_55eb19bd-ec6a-4a32-a1cf-78dee875fef6.gms Start 04/09/24 23:09:34 46.4.0 2cbe6e3b WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\pspkm\AppData\Local\Programs\Python\Python312\Lib\site-packages\gamspy_base\gmsprmNT.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP CONOPT
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS CONOPT
    DNLP CONOPT
    RMINLP CONOPT
    MINLP SBB
    QCP CONOPT
    MIQCP SBB
    RMIQCP CONOPT
    EMP CONVERT
    Restart C:\Users\pspkm\AppData\Local\Temp\tmp90y88ub_\_save_b5fc5b61-1bb9-42d5-98cd-974d4c2291b2.g00
    Input C:\Users\pspkm\AppData\Local\Temp\tmp90y88ub_\_job_55eb19bd-ec6a-4a32-a1cf-78dee875fef6.gms
    Output C:\Users\pspkm\AppData\Local\Temp\tmp90y88ub_\_job_55eb19bd-ec6a-4a32-a1cf-78dee875fef6.lst
    Save C:\Users\pspkm\AppData\Local\Temp\tmp90y88ub_\_restart_b5fc5b61-1bb9-42d5-98cd-974d4c2291b2.g00
    ScrDir C:\Users\pspkm\AppData\Local\Temp\tmp90y88ub_\225a\
    SysDir C:\Users\pspkm\

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalLocal,0.000458710425387965,3,57,NLP,CONOPT,0.015


In [104]:
w.records

,i,level,marginal,lower,upper,scale
0,0,0.000000,0.001065,0.0,inf,1.0
1,1,0.000000,0.000579,0.0,inf,1.0
2,2,0.000000,0.001226,0.0,inf,1.0
3,3,0.016916,-0.000000,0.0,inf,1.0
4,4,0.000000,0.000499,0.0,inf,1.0
5,5,0.000000,0.000518,0.0,inf,1.0
6,6,0.000000,0.000990,0.0,inf,1.0
7,7,0.000000,0.000804,0.0,inf,1.0
8,8,0.037197,-0.000000,0.0,inf,1.0
9,9,0.000000,0.001645,0.0,inf,1.0
